In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from tqdm import tqdm
import random

# 헤더 설정 (기존 headers 변수 사용)
# 대량 크롤링 시 IP 차단당해서 Client Error가 발생할 수 있으니 주기적으로 IP를 변경해주기
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.7151.104 Safari/537.36'
}

In [ ]:
# 크롤링 페이지 분류
start_no = 20250001
END_NO = 20250345


# Funtion1: 단일 게임 데이터를 크롤링
def crawl_single_game_data(s_no):
    url = f"https://statiz.sporki.com/schedule/?m=preview&s_no={s_no}"
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        total_score = crawl_score_from_soup(soup)
        pitchers = crawl_pitcher_from_soup(soup)
        game_date = crawl_date_from_soup(soup)
        
        return {
            's_no': s_no,
            '경기 날짜': game_date,
            '총득점': total_score,
            '홈 선발투수': pitchers[0] if len(pitchers) > 0 else None,
            '원정 선발투수': pitchers[1] if len(pitchers) > 1 else None
        }
        
    except Exception as e:
        print(f"s_no {s_no} 크롤링 실패: {e}")
        return None

# Function2: 득점 데이터 크롤링
def crawl_score_from_soup(soup):
    try:
        div_nums = soup.select('div.num')
        if not div_nums:
            return 0
        
        total_score = 0
        for div_num in div_nums:
            spans = div_num.find_all('span')
            for span in spans:
                text = span.get_text(strip=True)
                if text:
                    try:
                        num_value = float(text.replace(',', ''))
                        total_score += num_value
                    except ValueError:
                        continue
        return total_score
        
    except Exception as e:
        print(f"득점 크롤링 오류: {e}")
        return 0

# Function3: 투수 정보 크롤링
def crawl_pitcher_from_soup(soup):
    try:
        pitchers_name = [div.get_text(strip=True) for div in soup.find_all('div', style="margin-top:.3rem;")]
        return pitchers_name
    except Exception as e:
        print(f"투수 정보 크롤링 오류: {e}")
        return []

# Function4: 경기 날짜 크롤링 
def crawl_date_from_soup(soup):
    try:
        text = soup.select_one('div.txt').get_text(strip=True)
        date_data = re.search(r'\d{2}-\d{2}', text).group()
        return date_data
    except Exception as e:
        print(f"날짜 크롤링 오류: {e}")
        return None

# 전체 데이터 크롤링 함수 (DataFrame append 방식)
def crawl_all_games():
    # 빈 DataFrame 초기화
    merged_df = pd.DataFrame(columns=['s_no', '경기 날짜', '총득점', '홈 선발투수', '원정 선발투수'])
    
    for s_no in tqdm(range(start_no, END_NO + 1), desc="크롤링 진행"):
        game_data = crawl_single_game_data(s_no)
        if game_data:
            # 개별 게임 데이터를 DataFrame으로 변환
            single_game_df = pd.DataFrame([game_data])
            # 전체 DataFrame에 append
            merged_df = pd.concat([merged_df, single_game_df], ignore_index=True)
            print(f"s_no {s_no} 추가 완료 (총 {len(merged_df)}개)")
        
        # 서버 부하 방지를 위한 딜레이
        # 초기에 0.5로 설정하니까 웹사이트 내부에서 비정상적인 행동 패턴으로 파악하고 크롤링을 차단해버림
        time.sleep(random.uniform(1.5, 3.0))
    
    return merged_df

# 에러 처리 및 재시도 기능 포함 크롤링 (DataFrame append 방식)
def crawl_with_retry(start_no=start_no, end_no=END_NO, max_retries=3):
    # 빈 DataFrame 초기화
    merged_df = pd.DataFrame(columns=['s_no', '경기 날짜', '총득점', '홈 선발투수', '원정 선발투수'])
    failed_s_nos = []
    
    for s_no in tqdm(range(start_no, end_no + 1), desc="크롤링 진행"):
        success = False
        for attempt in range(max_retries):
            try:
                game_data = crawl_single_game_data(s_no)
                if game_data:
                    # 개별 게임 데이터를 DataFrame으로 변환
                    single_game_df = pd.DataFrame([game_data])
                    # 전체 DataFrame에 append
                    merged_df = pd.concat([merged_df, single_game_df], ignore_index=True)
                    print(f"s_no {s_no} 추가 완료 (총 {len(merged_df)}개)")
                    success = True
                    break
                else:
                    time.sleep(1)  # 실패 시 잠시 대기
            except Exception as e:
                if attempt == max_retries - 1:
                    print(f"s_no {s_no} 최종 실패: {e}")
                    failed_s_nos.append(s_no)
                else:
                    time.sleep(2)  # 재시도 전 더 긴 대기
        if success:
            time.sleep(0.5)  # 성공 시 기본 대기
    if failed_s_nos:
        print(f"실패한 s_no 목록: {failed_s_nos}")
    
    return merged_df, failed_s_nos

# 중간 저장 기능 포함 크롤링 (DataFrame append 방식, 추천)
def crawl_with_checkpoint(start_no=start_no, end_no=END_NO, checkpoint_interval=50):
    # 빈 DataFrame 초기화
    merged_df = pd.DataFrame(columns=['s_no', '경기 날짜', '총득점', '홈 선발투수', '원정 선발투수'])
    
    for i in range(start_no, end_no + 1, checkpoint_interval):
        batch_end = min(i + checkpoint_interval - 1, end_no)
        for s_no in tqdm(range(i, batch_end + 1), desc=f"배치 {i}-{batch_end}"):
            game_data = crawl_single_game_data(s_no)
            if game_data:
                # 개별 게임 데이터를 DataFrame으로 변환
                single_game_df = pd.DataFrame([game_data])
                # 전체 DataFrame에 append
                merged_df = pd.concat([merged_df, single_game_df], ignore_index=True)
            time.sleep(0.5)
        # 중간 저장 (백업용)
        checkpoint_file = f'checkpoint_baseball_data_{start_no}_games.csv'
        merged_df.to_csv(checkpoint_file, index=False, encoding='utf-8-sig')
    
    return merged_df

# 실패한 s_no들에 대해서만 재크롤링 (DataFrame append 방식)
def retry_failed_games(failed_s_nos, max_retries=5):
    if not failed_s_nos:
        print("재크롤링할 데이터가 없습니다.")
        return pd.DataFrame(), []
    # 빈 DataFrame 초기화
    retry_df = pd.DataFrame(columns=['s_no', '경기 날짜', '총득점', '홈 선발투수', '원정 선발투수'])
    still_failed = []
    
    for s_no in tqdm(failed_s_nos, desc="재크롤링"):
        success = False
        for attempt in range(max_retries):
            try:
                game_data = crawl_single_game_data(s_no)
                if game_data:
                    # 개별 게임 데이터를 DataFrame으로 변환
                    single_game_df = pd.DataFrame([game_data])
                    # 재시도 DataFrame에 append
                    retry_df = pd.concat([retry_df, single_game_df], ignore_index=True)
                    success = True
                    break
                else:
                    time.sleep(2)
            except Exception as e:
                time.sleep(3)
        if not success:
            still_failed.append(s_no)
    if still_failed:
        print(f"여전히 실패한 s_no: {still_failed}")
    
    return retry_df, still_failed


merged_df = crawl_with_checkpoint(start_no, END_NO, checkpoint_interval=50)


배치 20250355-20250355: 100%|██████████| 1/1 [00:01<00:00,  1.23s/it]
